In [2]:
import math
from scripts_stock.cfg.out_file_name import OutFileName
from scripts_stock.cfg.set_dir import ProjectDir
from scripts_stock.cfg.stock_list import *
from scripts_stock.utils.analysis.stock_stat_index import df_to_stock_df, stock_kdj
import os
from scripts_stock.utils.common import CommonScript
from scripts_stock.utils.string_process import StringProcess


In [25]:
pd.set_option('display.max_rows', None)  # 显示所有行
pd.set_option('display.max_columns', None)  # 显示所有列
pd.set_option('display.width', None)  # 自动调整列宽

In [5]:
def get_data_sql_str_macd():
    return f"""SELECT *  FROM t_stock_kdj_daily_all where date>='2022-01-01'  """

In [6]:
conn = CommonScript.connect_to_db("test.db")
cursor = conn.cursor()
df1 = pd.read_sql_query(get_data_sql_str_macd(),conn)

In [37]:
df2 = df1[df1["stock_index"]==600011][["date","stock_index","kdjj"]]

In [34]:
def subtract_and_check(row):
    if row['kdjj_quant_5'] - row['kdjj'] > 0:
        return 1
    else:
        return 0

In [32]:
df2["kdjj_quant_5"]-df2["kdjj"]

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
5            NaN
6            NaN
7            NaN
8            NaN
9            NaN
10           NaN
11           NaN
12           NaN
13           NaN
14           NaN
15           NaN
16           NaN
17           NaN
18           NaN
19           NaN
20           NaN
21           NaN
22           NaN
23           NaN
24           NaN
25           NaN
26           NaN
27           NaN
28           NaN
29           NaN
30           NaN
31           NaN
32           NaN
33           NaN
34           NaN
35           NaN
36           NaN
37           NaN
38           NaN
39           NaN
40           NaN
41           NaN
42           NaN
43           NaN
44           NaN
45           NaN
46           NaN
47           NaN
48           NaN
49           NaN
50           NaN
51           NaN
52           NaN
53           NaN
54           NaN
55           NaN
56           NaN
57           NaN
58           N

In [38]:
import numpy as np
df2["kdjj_quant_5"] = df2['kdjj'].rolling(180, min_periods=180).quantile(0.05)
df2["kdjj_quant_95"] = df2['kdjj'].rolling(180, min_periods=180).quantile(0.95)
df2['kdjj_below_180d_quan5'] = df2.apply(subtract_and_check, axis=1)

In [52]:
data_list = []
for stock_in in df1["stock_index"].unique().tolist():
    df2 = df1[df1["stock_index"]==stock_in][["date","stock_index","kdjj","macd","close"]]
    df2["kdjj_quant_5"] = df2['kdjj'].rolling(180, min_periods=180).quantile(0.05)
    df2["kdjj_quant_95"] = df2['kdjj'].rolling(180, min_periods=180).quantile(0.95)
    df2['kdjj_below_180d_quan5'] = df2.apply(subtract_and_check, axis=1)
    data_list.append(df2)

df3 = pd.concat(data_list)


In [55]:
df3[(df3["kdjj_below_180d_quan5"]==1)&(df3["date"]>="2024-11-01")&(df3["macd"]<0)&(df3["close"]<30)].sort_values("date")

,date,stock_index,kdjj,macd,close,kdjj_quant_5,kdjj_quant_95,kdjj_below_180d_quan5
132687,2024-11-12,601225,-7.906,-0.239,23.16,-2.38420,101.01700,1
71234,2024-11-14,600989,-10.948,-0.021,15.46,-1.09525,103.24615,1
19561,2024-11-14,601899,-8.284,-0.195,15.61,-2.42130,100.82480,1
145270,2024-11-14,603993,-6.698,-0.026,7.44,-6.41820,103.77620,1
50290,2024-11-14,601169,-6.205,-0.003,5.56,-5.92570,107.21960,1
145271,2024-11-15,603993,-11.524,-0.055,7.46,-6.56215,103.77620,1
133389,2024-11-15,600795,-4.494,-0.065,4.76,-1.51745,101.86145,1
150164,2024-11-15,600000,-8.386,-0.004,9.54,-1.59855,102.32160,1
19562,2024-11-15,601899,-9.539,-0.261,15.62,-2.72265,100.38625,1
145272,2024-11-18,603993,-10.680,-0.081,7.40,-6.69940,103.77620,1
